# Harmonica Training on Google Colab

This notebook sets up and trains the Harmonica voice cloning system on Colab.

**Requirements:**
- Google Colab with GPU runtime (T4 or better)
- Google Drive for checkpoint storage

## 1. Setup

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone repository (or copy from Drive)
!git clone https://github.com/yourusername/harmonica.git
%cd harmonica

In [ ]:
# Install dependencies
!pip install -e ".[dev]" -q

In [ ]:
# Verify installation
import torch
from harmonica.utils.device import device_info

print(f"PyTorch version: {torch.__version__}")
print(f"Device info: {device_info()}")

## 2. Download and Preprocess Data

In [ ]:
# Download LJSpeech dataset
!python scripts/download_data.py --dataset ljspeech --output-dir ./data

In [ ]:
# Preprocess (encode audio to codec tokens)
!python scripts/preprocess.py --dataset ljspeech --data-dir ./data/LJSpeech-1.1 --output-dir ./cache/ljspeech

## 3. Training

In [ ]:
# Set up checkpoint directory on Google Drive for persistence
import os
os.makedirs('/content/drive/MyDrive/harmonica_checkpoints', exist_ok=True)
!ln -sf /content/drive/MyDrive/harmonica_checkpoints ./experiments/checkpoints

In [ ]:
# Train AR model
!python scripts/train.py \
    --config configs/experiment/baseline.yaml \
    --model-type ar \
    --device cuda

In [ ]:
# Resume training after disconnect
!python scripts/train.py \
    --config configs/experiment/baseline.yaml \
    --model-type ar \
    --device cuda \
    --resume

## 4. Generate Samples

In [ ]:
# Synthesize speech
!python scripts/synthesize.py \
    --text "Hello, this is a test of the voice cloning system." \
    --ar-checkpoint experiments/checkpoints/checkpoint_best.pt \
    --output test_output.wav

In [ ]:
# Play audio
from IPython.display import Audio
Audio('test_output.wav')

## 5. Evaluation

In [ ]:
# Run evaluation on test sentences
!python scripts/evaluate.py \
    --ar-checkpoint experiments/checkpoints/checkpoint_best.pt \
    --output-dir ./eval/output

In [ ]:
# View results
import json
with open('eval/output/results.json') as f:
    results = json.load(f)
print(json.dumps(results['statistics'], indent=2))

## 6. Copy Checkpoints to Drive

In [ ]:
# Checkpoints are automatically saved to Drive via symlink
# List checkpoints
!ls -la /content/drive/MyDrive/harmonica_checkpoints/